In [1]:
import nltk
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\User\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [3]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.append("product")
stop_words.append("item")

# First round ot topic analysis

In [4]:
data_file = "../Data/all_data_clean.csv"
df = pd.read_csv(data_file)
df = df[df["Review_splitted"] != "no comments review is an image" ]
df = df[df["Review_splitted"] != "no review receive" ]
df = df[df["Review_splitted"] != "your review has been hidden due to inappropriate content note hope may remove the coins awarded for this review" ]
print(len(df))


4971


,Unnamed: 0,Platform,Brand,Category,Product Name,Price,Reviewer,Review,Review_splitted,Product Purchase,Ratings,Date Of Review,Response
0,0,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,zenheng,"delivery was quite fast, and item was on disco...",delivery was quite fast and item was on discou...,NaN,4.0,2020-02-06 23:58:00,no
1,1,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,glennoeestaquio,received item sealed properly.\r\r\r\nquick de...,received item sealed properly quick delivery t...,NaN,5.0,2020-02-06 12:44:00,no
2,2,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,shy2206,yet to try but delivery was prompt and product...,yet to try but delivery was prompt and product...,NaN,5.0,2020-02-06 10:21:00,no
3,3,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,lindamiyalin,"well wrapped, in good condition, super fast ...",well wrapped in good condition super fast deli...,NaN,5.0,2020-01-30 23:01:00,no
4,4,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,kyc385,"well packed, will buy again",well packed will buy again,NaN,5.0,2020-02-01 11:40:00,no
5,5,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,junicaj,well wrap in good condition,well wrap in good condition,NaN,5.0,2020-02-01 23:25:00,no
6,6,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,aciirt,came in good condition. will start using soon....,came in good condition will start using soon t...,NaN,5.0,2020-01-26 22:21:00,no
7,7,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,gm.pub,soothing feel,soothing feel,NaN,5.0,2020-01-30 15:45:00,no
8,8,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,ojmelt,"super fast delivery, received item the followi...",super fast delivery received item the followin...,NaN,5.0,2020-01-24 18:59:00,no
9,9,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,areumam,effective product. fast delivery. received 1 d...,effective product fast delivery received 1 day...,NaN,5.0,2020-01-26 12:42:00,no


In [16]:
doc_id = df['Unnamed: 0']
doc_id = (doc_id.tolist())
print(len(doc_id))

4971


In [17]:
data = df.Review.values.tolist()


In [18]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['delivery', 'was', 'quite', 'fast', 'and', 'item', 'was', 'on', 'discount', 'too', 'so', 'couldn', 'complain', 'so', 'far', 'liking', 'this', 'product', 'will', 'continue', 'to', 'use', 'it']]


In [19]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['delivery', 'was', 'quite', 'fast', 'and', 'item', 'was', 'on', 'discount', 'too', 'so', 'couldn', 'complain', 'so', 'far', 'liking', 'this', 'product', 'will', 'continue', 'to', 'use', 'it']


In [20]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]


def lemmatization(texts, allowed_postags=['NOUN', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [21]:
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams)

print(data_lemmatized[:1])

[['delivery', 'quite', 'discount', 'complain', 'far', 'like', 'continue', 'use']]


In [22]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]]


In [23]:
import datetime
print(datetime.datetime.now())

from gensim.models import CoherenceModel

model_list = []
coherence_values = []
model_topics = []

for num_topics in range(2, 10):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto')
    
    coherencemodel = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    model_topics.append(num_topics)
    model_list.append(lda_model)
    coherence_values.append(coherencemodel.get_coherence())
    print("#Topics: " + str(num_topics) + " Score: " + str(coherencemodel.get_coherence()))

print(datetime.datetime.now())

2020-02-20 23:09:51.756189
#Topics: 2 Score: 0.5079442162129209
#Topics: 3 Score: 0.4937683419550018
#Topics: 4 Score: 0.4748157466263377
#Topics: 5 Score: 0.4091475629265945
#Topics: 6 Score: 0.4005046905919922
#Topics: 7 Score: 0.3935751051783603
#Topics: 8 Score: 0.4245477059988087
#Topics: 9 Score: 0.3711368277034464
2020-02-20 23:13:03.315382


In [ ]:
import matplotlib.pyplot as plt

limit=10; start=2; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10)
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

In [ ]:
#Find most dominant topic

import numpy as np
import pandas as pd

def format_topics_sentences(ldamodel, corpus, data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


#I choose model_list[1] where the number of topics is 4
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, data=data_lemmatized)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']


# Show
df_dominant_topic.head(100)


In [ ]:
data_file = "../Data/all_data_raw.csv"
data = pd.read_csv(data_file)
data.columns = data.columns.str.strip().str.replace(" ","_") # reason for doing this is cause cannot get columns with space into a list

topic = df_dominant_topic["Dominant_Topic"]
topic_word = []
for t in topic:
    if t == 0.0:
        topic_word.append("product")
    elif t == 1.0:
        topic_word.append("after use")
    elif t == 2.0:
        topic_word.append("no comments")
    else:
        topic_word.append("delivery")
        
data["Dorminant Topic"] = topic
data["topic"] = topic_word
data

data.to_csv("data_with_topic.csv")

In [ ]:
data_file = "../Data/all_data_clean.csv"
data = pd.read_csv(data_file)
data.columns = data.columns.str.strip().str.replace(" ","_") # reason for doing this is cause cannot get columns with space into a list
raw_review = data["Review"]
df_dominant_topic["raw review"] = raw_review
df_dominant_topic.to_csv("result_4topics.csv")

# LDA Mallet

In [ ]:
import gensim
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
import os
os.environ.update({'MALLET_HOME':r'D:/Softwares/mallet/mallet-2.0.8/'})

no_topics=10

mallet_path = r'D:\Softwares\mallet\mallet-2.0.8\bin\mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=3, id2word=id2word)

pprint(ldamallet.show_topics(formatted=False))